# 1. 분류_당뇨병_머신러닝

## (1) import

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

# 전처리
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 모델
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression

# 예측
from sklearn.metrics import accuracy_score, classification_report

# 모델 저장
import pickle

## (2) 데이터 확인

In [3]:
# folder_path = ''
folder_path='/content/drive/MyDrive/고수의클래스_0906/1교시_Tabular/분류_당뇨병_머신러닝/data_folder'

In [36]:
# train = pd.read_csv(folder_path + 'train.csv')
# test = pd.read_csv(folder_path + 'test.csv')
train = pd.read_csv(folder_path + '/diabetes_train.csv')
test = pd.read_csv(folder_path + '/diabetes_testset30.csv')

In [37]:
display(train.head())
display(test.head())

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Sex,Outcome
0,6,148,72,35,0,33.6,0.627,50,Male,1
1,1,85,66,29,0,26.6,0.351,31,Female,0
2,8,183,64,0,0,23.3,0.672,32,Male,1
3,1,89,66,23,94,28.1,0.167,21,Female,0
4,0,137,40,35,168,43.1,2.288,33,Female,1


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Sex
0,2,99,60,17,160,36.6,0.453,21,Female
1,1,102,74,0,0,39.5,0.293,42,Male
2,11,120,80,37,150,42.3,0.785,48,Male
3,3,102,44,20,94,30.8,0.400,26,Male
4,1,109,58,18,116,28.5,0.219,22,Male


In [38]:
## Feature 와 Label 분리
X_train = train.drop(columns=["Outcome"])
y = train["Outcome"]

## (3) 데이터 전처리
- 결측치 제거는 시험에 잘 안 나온다고 함
- 정규화/표준화 안 해도 되긴 함

### 1. 레이블링 ( 문자 -> 숫자로 인코딩 )

In [39]:
encoder = LabelEncoder()
X_train['Sex'] = encoder.fit_transform(X_train['Sex'])
test['Sex'] = encoder.transform(test['Sex']) # test 데이터는 fit_transform이 아니라 transform만 해준다. 왜지? fit은 train 데이터에만 적용해야 하기 때문이다.

### 2. 스케일링 ( 표준화 )

In [40]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
test = scaler.transform(test)

## (4) 학습

In [41]:
# train, validation 분류
X_train, X_test, y_train, y_test = train_test_split(X_train, y, test_size=0.2, random_state=42)

In [42]:
# 학습
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

## (5) 예측 및 평가

In [43]:
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"모델 정확도: {accuracy}")
print("분류 보고서:\n", report)

모델 정확도: 0.7702702702702703
분류 보고서:
               precision    recall  f1-score   support

           0       0.78      0.88      0.82        91
           1       0.76      0.60      0.67        57

    accuracy                           0.77       148
   macro avg       0.77      0.74      0.75       148
weighted avg       0.77      0.77      0.76       148



## (6) test 데이터 예측

In [44]:
filename = "01073002902_1.h5"

pickle.dump(model, open(filename, 'wb'))

In [46]:
pred = model.predict(test)
print('pred:', pred)

pred: [0 0 1 0 0 1 1 0 1 0 1 1 0 0 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0]


In [48]:
test = pd.read_csv(folder_path + '/diabetes_testset30.csv')
test["Outcome"] = pred

## (7) 결과 제출

In [49]:
test.to_csv("01073002902_1.csv", index=False)

# 2. 분류_타이타닉생존자_신경망_머신러닝

https://pinkwink.kr/1119 참고

# (1) import

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 학습
from sklearn.model_selection import train_test_split

# 신경망 모델
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Reshape, Permute

# 머신러닝 모델
from sklearn.linear_model import LogisticRegression

# 평가
from sklearn.metrics import accuracy_score, classification_report

## (2) 데이터 확인

In [75]:
# folder_path = ''
folder_path = '/content/drive/MyDrive/고수의클래스_0906/1교시_Tabular/분류_타이타닉생존자_신경망_머신러닝/data_folder'

In [76]:
# train = pd.read_csv(folder_path + 'train.csv')
# test = pd.read_csv(folder_path + 'test.csv')
train = pd.read_excel(folder_path + '/titanic_train.xls')
test = pd.read_excel(folder_path + '/titanic_test30.xls')

In [77]:
display(train.head())
display(test.head())

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,3,0,"Vestrom, Miss. Hulda Amanda Adolfina",female,14.0,0,0,350406,7.8542,NaN,S,NaN,NaN,NaN
1,3,0,"Vovk, Mr. Janko",male,22.0,0,0,349252,7.8958,NaN,S,NaN,NaN,NaN
2,3,0,"Waelens, Mr. Achille",male,22.0,0,0,345767,9.0000,NaN,S,NaN,NaN,"Antwerp, Belgium / Stanton, OH"
3,3,0,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,NaN,NaN,NaN
4,3,0,"Warren, Mr. Charles William",male,NaN,0,0,C.A. 49867,7.5500,NaN,S,NaN,NaN,NaN


## (2) 데이터 전처리

### 1. 문자 -> 숫자 인코딩

In [78]:
train['sex'] = train['sex'].map({'female': 1, 'male': 0})
test['sex'] = test['sex'].map({'female': 1, 'male': 0})

### 2. int -> float 형으로 변환 ( 왜지? )

In [79]:
cols_to_float = ['survived', 'pclass', 'sex', 'sibsp', 'parch', 'fare']
train[cols_to_float] = train[cols_to_float].astype('float')

test[cols_to_float] = test[cols_to_float].astype('float')

### 3. Null 값이 있는 행 삭제

In [80]:
train = train.dropna(subset=['age', 'sibsp', 'parch', 'fare'])
test = test.dropna(subset=['age', 'sibsp', 'parch', 'fare'])

In [81]:
display(train.info())
display(test.info())

<class 'pandas.core.frame.DataFrame'>
Index: 1024 entries, 0 to 1277
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     1024 non-null   float64
 1   survived   1024 non-null   float64
 2   name       1024 non-null   object 
 3   sex        1024 non-null   float64
 4   age        1024 non-null   float64
 5   sibsp      1024 non-null   float64
 6   parch      1024 non-null   float64
 7   ticket     1024 non-null   object 
 8   fare       1024 non-null   float64
 9   cabin      272 non-null    object 
 10  embarked   1022 non-null   object 
 11  boat       415 non-null    object 
 12  body       112 non-null    float64
 13  home.dest  684 non-null    object 
dtypes: float64(8), object(6)
memory usage: 120.0+ KB


None

<class 'pandas.core.frame.DataFrame'>
Index: 20 entries, 0 to 29
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     20 non-null     float64
 1   survived   20 non-null     float64
 2   name       20 non-null     object 
 3   sex        20 non-null     float64
 4   age        20 non-null     float64
 5   sibsp      20 non-null     float64
 6   parch      20 non-null     float64
 7   ticket     20 non-null     object 
 8   fare       20 non-null     float64
 9   cabin      0 non-null      float64
 10  embarked   20 non-null     object 
 11  boat       2 non-null      object 
 12  body       7 non-null      float64
 13  home.dest  1 non-null      object 
dtypes: float64(9), object(5)
memory usage: 2.3+ KB


None

## (3) 학습

In [104]:
# train 데이터에서 feature와 label 분리 ( 필요한 컬럼만 추출: pclass, sex, age, sibsp, parch, fare )
X_train = train.values[:, [0,3,4,5,6,8]]
y = train["survived"]

test_data = test.values[:, [0,3,4,5,6,8]]

In [101]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y, test_size=0.1, random_state=7)

### 1. 신경망

In [90]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((921, 6), (103, 6), (921,), (103,))

In [91]:
model = Sequential()
model.add(Dense(255, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='mse', optimizer='Adam', metrics=['accuracy'])
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                      │ (None, 255)                 │           1,785 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │             256 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,041 (7.97 KB)

 Trainable params: 2,041 (7.97 KB)

 Non-trainable params: 0 (0.00 B)

In [92]:
X_train = np.array(X_train, dtype=np.float32)
y_train = np.array(y_train, dtype=np.float32)
X_test = np.array(X_test, dtype=np.float32)
y_test = np.array(y_test, dtype=np.float32)

### 2. 머신러닝

In [105]:
y_binary = y.astype(int)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_binary, test_size=0.1, random_state=7)

In [109]:
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

LogisticRegression()

## (4) 예측 및 평가

In [107]:
# 신경망
hist = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=500)

plt.figure(figsize=(12, 8))
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.legend(['loss', 'val_loss', 'accuracy', 'val_accuracy'])
plt.show()

TypeError: LogisticRegression.fit() got an unexpected keyword argument 'validation_data'

In [110]:
# 머신러닝
y_pred = logistic_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"모델 정확도: {accuracy}")
print("분류 보고서:\n", report)

모델 정확도: 0.7281553398058253
분류 보고서:
               precision    recall  f1-score   support

           0       0.75      0.80      0.77        59
           1       0.70      0.64      0.67        44

    accuracy                           0.73       103
   macro avg       0.72      0.72      0.72       103
weighted avg       0.73      0.73      0.73       103



## (5) test 데이터 예측

In [94]:
# 신경망
result = []

for item in test_data:
    item = np.array(item, dtype=np.float32)
    pred = model.predict(item.reshape(1, 6))
    if pred >= 0.5:
        pred = 1
    else:
        pred = 0
    result.append(pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


In [95]:
result

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [96]:
test_data

array([[3.0, 1.0, 14.0, 0.0, 0.0, 7.8542],
       [3.0, 0.0, 22.0, 0.0, 0.0, 7.8958],
       [3.0, 0.0, 22.0, 0.0, 0.0, 9.0],
       [3.0, 0.0, 32.5, 0.0, 0.0, 9.5],
       [3.0, 1.0, 38.0, 0.0, 0.0, 7.2292],
       [3.0, 0.0, 51.0, 0.0, 0.0, 7.75],
       [3.0, 0.0, 18.0, 1.0, 0.0, 6.4958],
       [3.0, 0.0, 21.0, 1.0, 0.0, 6.4958],
       [3.0, 1.0, 47.0, 1.0, 0.0, 7.0],
       [3.0, 0.0, 28.5, 0.0, 0.0, 16.1],
       [3.0, 0.0, 21.0, 0.0, 0.0, 7.25],
       [3.0, 0.0, 27.0, 0.0, 0.0, 8.6625],
       [3.0, 0.0, 36.0, 0.0, 0.0, 9.5],
       [3.0, 0.0, 27.0, 1.0, 0.0, 14.4542],
       [3.0, 1.0, 15.0, 1.0, 0.0, 14.4542],
       [3.0, 0.0, 45.5, 0.0, 0.0, 7.225],
       [3.0, 1.0, 14.5, 1.0, 0.0, 14.4542],
       [3.0, 0.0, 26.5, 0.0, 0.0, 7.225],
       [3.0, 0.0, 27.0, 0.0, 0.0, 7.225],
       [3.0, 0.0, 29.0, 0.0, 0.0, 7.875]], dtype=object)

In [97]:
# 신경망
test['result'] = result

In [112]:
# 머신러닝
y_pred = logistic_model.predict(test_data)

In [113]:
y_pred

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0])

In [114]:
# 머신러닝
test['result'] = y_pred

## (6) 결과 제출

In [98]:
pickle.dump(model, open('01073002902_1.h5', 'wb'))
test.to_csv('01073002902_1.csv', index=False)

# 3. 회귀_당뇨병

## (1) import

In [115]:
import pandas as pd

# 학습
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# 평가
from sklearn.metrics import mean_squared_error
from math import sqrt

# 제출
import pickle

## (2) 데이터 확인

In [116]:
# folder_path = ''
folder_path='/content/drive/MyDrive/고수의클래스_0906/1교시_Tabular/회귀_당뇨병_머신러닝/data_folder_reg'

In [117]:
train = pd.read_csv(folder_path + '/diabetes_reg_train.csv')
test = pd.read_csv(folder_path + '/diabetes_reg_test30.csv')

In [118]:
display(train.head())
display(test.head())

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135


,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.074401,-0.044642,0.085408,0.063187,0.014942,0.013091,0.015505,-0.002592,0.006207,0.085907
1,-0.052738,-0.044642,-0.000817,-0.026328,0.010815,0.007141,0.048640,-0.039493,-0.035816,0.019633
2,0.081666,0.050680,0.006728,-0.004534,0.109883,0.117056,-0.032356,0.091875,0.054720,0.007207
3,-0.005515,-0.044642,0.008883,-0.050427,0.025950,0.047224,-0.043401,0.071210,0.014821,0.003064
4,-0.027310,-0.044642,0.080019,0.098751,-0.002945,0.018101,-0.017629,0.003312,-0.029526,0.036201


## (3) 데이터 전처리

In [ ]:
## 숫자 인코딩 되어 있음?
## 스케일링 이미 되어 있음?
## -> 이러면 pass

## (4) 학습

In [119]:
# train 데이터에서 feature와 label 분리
X_train = train.drop(columns=["target"])
y = train["target"]
display(X_train.head())
display(y.head())

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641


,target
0,151
1,75
2,141
3,206
4,135


In [120]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y, test_size=0.3, random_state=42)

In [121]:
# 다중선형회귀 모델
multi_regressor = LinearRegression()
multi_regressor.fit(X_train, y_train)

LinearRegression()

## (5) 평가

In [122]:
multi_test_pred = multi_regressor.predict(X_test)

In [123]:
# 학습시킨 모델에 test(정확히는 validation)를 넣어 예측
# 이후 예측값과 실제값을 비교하여 모델의 성능을 평가
multi_test_mse = mean_squared_error(multi_test_pred, y_test)

In [124]:
# 기준에 만족하면 모델 저장
print(f"Multi Regression Test RMSE is {sqrt(multi_test_mse)}")

Multi Regression Test is 55.71399271574175


## (6) 결과 제출

In [125]:
pickle.dump(multi_regressor, open('01073002902_1.h5', 'wb'))

In [126]:
# 진짜 test 데이터 예측
multi_test_pred = multi_regressor.predict(test)

In [128]:
test['target'] = multi_test_pred
test.to_csv('01073002902_1.csv', index=False)

In [129]:
test.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.074401,-0.044642,0.085408,0.063187,0.014942,0.013091,0.015505,-0.002592,0.006207,0.085907,240.298905
1,-0.052738,-0.044642,-0.000817,-0.026328,0.010815,0.007141,0.048640,-0.039493,-0.035816,0.019633,117.674270
2,0.081666,0.050680,0.006728,-0.004534,0.109883,0.117056,-0.032356,0.091875,0.054720,0.007207,165.673517
3,-0.005515,-0.044642,0.008883,-0.050427,0.025950,0.047224,-0.043401,0.071210,0.014821,0.003064,172.428616
4,-0.027310,-0.044642,0.080019,0.098751,-0.002945,0.018101,-0.017629,0.003312,-0.029526,0.036201,236.235847
